# Topic modeling (librería Gensim)
Vamos a ver cómo realizar un modelado de temática en grandes volúmenes de texto con la librería `gensim`  

Utilizaremos el conjunto de datos *Lee* de `Gensim` (es una versión abreviada del conjunto http://www.socsci.uci.edu/~mdlee/lee_pincombe_welsh_document.PDF).  

Para visualizar gráficamente los tópicos es necesario instalar la librería `pyLDAvis` dentro del entorno de Anaconda con el comando:
```python
conda install -c conda-forge pyldavis 
```

### Cargamos librerías

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from pprint import pprint
import warnings

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning)


# spacy para lematizar
import spacy

# herramientas de dibujado
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

Utilizamos un generador para obtener los documentos del Corpus línea a línea desde el archivo del conjunto de ejemplo y convertirlos en un listado de tokens.

In [ ]:
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])
stop_words = nlp.Defaults.stop_words #listado de stop-words

def lemmatize_doc(text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV', 'PROPN']):
    """Función que devuelve el lema de una string,
    excluyendo las palabras cuyo POS_TAG no está en la lista"""
    text_out = [t.lemma_.lower() for t in nlp(text)
                if t.pos_ in allowed_postags
                and len(t.lemma_)>3
                and not t.is_stop]
    return text_out
            
def build_texts(fname):
    """
    Generador que devuelve el texto tokenizado a partir de un archivo
    línea a línea
    """
    with open(fname) as f:
        for line in f:
            yield lemmatize_doc(line)

In [ ]:
lee_data_file = 'lee_background.cor'

In [ ]:
with open(lee_data_file) as f:
        for line in f:
            print(line)
            break

In [ ]:
texto=build_texts(lee_data_file)

In [ ]:
texto

In [ ]:
texto1 = next(texto)
print(texto1)

In [ ]:
for c in texto:
    print(c)
    break

In [ ]:
lista_procesado = [c for c in texto]

In [ ]:
len(lista_procesado)

In [ ]:
texto = build_texts(lee_data_file)
corpus = [c for c in texto]

In [ ]:
len(corpus)

Creamos el diccionario usando el generador para no cargar el corpus completo en memoria. Luego lo usamos para generar el BoW directamente desde el archivo

In [ ]:
# Crea diccionario
diccionario = corpora.Dictionary(build_texts(lee_data_file))
# Crea corpus (BoW)
corpus = [diccionario.doc2bow(text) for text in build_texts(lee_data_file)]

# Vemos como ejemplo el primer doc
print(corpus[0])

In [ ]:
len(corpus[0])

In [ ]:
len(corpus)

In [ ]:
diccionario.num_docs

In [ ]:
len(diccionario.items())

In [ ]:
warnings.filterwarnings('ignore')


ldamodel = LdaModel(corpus=corpus, num_topics=4, id2word=diccionario, iterations=5000)
pprint(ldamodel.print_topics())

In [ ]:
ldamodel[corpus]

In [ ]:
ldamodel[corpus[0]]

In [ ]:
ldamodel[corpus[201]]

### Visualización de los temas  
Podemos visualizarlo gráficamente la distribución de los documentos del Corpus por temas con la librería `pyLDAvis`

In [ ]:
vis_data = gensimvis.prepare(ldamodel, corpus, diccionario)
pyLDAvis.display(vis_data)

### Creamos bigramas y trigramas
Creamos un modelo para las palabras más frecuentes como bigrama o trigrama para considerar estos tokens juntos en lugar de separados.

In [ ]:
bigram = gensim.models.Phrases(build_texts(lee_data_file), min_count=5, threshold=50) # higher threshold fewer phrases.
#optimizamos una vez entreando
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [ ]:
bigram_mod

Por ejemplo los bigramas que ha encontrado para el documento con índice 1 son:

In [ ]:
print(bigram_mod[texto1])

In [ ]:
#creamos trigramas
trigram = gensim.models.Phrases(bigram_mod[build_texts(lee_data_file)], min_count=5, threshold=50)  
trigram_mod = gensim.models.phrases.Phraser(trigram)

def make_trigrams(text):
    '''Devuelve un doc convertido en trigramas según el
    modelo trigram_mod. La entrada tiene que ser una lista
    de de tokens'''
    return trigram_mod[bigram_mod[text]]

Para calcular los trigramas, aplicamos este modelo sobre la salida del modelo de bigramas:

In [ ]:
print(trigram_mod[bigram_mod[texto1]])

In [ ]:
print(make_trigrams(texto1))

Podemos ver los bigramas y trigramas que ha encontrado en el documento con una búsqueda de patrones regulares:

In [ ]:
trigram_sentence = ' '.join(make_trigrams(texto1))
re.findall(r'\w+_\w+', trigram_sentence)

Transformamos el corpus de texto con el modelo de trigramas. Usamos la función de tipo iterador `map` para no cargar todo el corpus intermedio en memoria.

In [ ]:
textos_trigramas = map(make_trigrams, build_texts(lee_data_file)) #aplica modelo trigramas

In [ ]:
type(textos_trigramas)

In [ ]:
print(next(textos_trigramas))

### Creamos el diccionario y el corpus para Topic Modeling
Las dos entradas para el modelo LDA son un diccionario de `gensim` y un corpus de texto.  
Preparamos el diccionario:

In [ ]:
# Crea diccionario
textos_trigramas = map(make_trigrams, build_texts(lee_data_file)) #aplica modelo trigramas
diccionario = corpora.Dictionary(textos_trigramas)
# Crea corpus (BoW)
textos_trigramas = map(make_trigrams, build_texts(lee_data_file)) #aplica modelo trigramas
corpus = [diccionario.doc2bow(text) for text in textos_trigramas]

# Vemos como ejemplo el primer doc
print(corpus[0])

In [ ]:
len(corpus[0])

In [ ]:
len(diccionario.token2id)

Recuerda que en el modelo BoW de `gensim` el primer elemento de cada tupla es el ID del término en el diccionario, y el segundo su frecuencia en el doc.  
`diccionario[ID]` devuelve el término con índice ID en el vocabulario:

In [ ]:
[(diccionario[id], freq) for id, freq in corpus[0]]

## Topic modeling

### Modelo LSI
Este modelo ordena los temas y saca un listado ordenado. Hay que especificar el número de topics.

Vamos a usar el algoritmo Latent Dirichlet Allocation (LDA) de `gensim` con la implementación multicore

In [ ]:
lsimodel = LsiModel(corpus=corpus, num_topics=4, id2word=diccionario)
pprint(lsimodel.show_topics())

In [ ]:
#aplicamos el modelo sobre el texto 2
lsimodel[corpus[2]]

### Modelo LDA
Es un modelo generativo que considera cada documento como una mezcla de temas donde cada tema tiene una distribución de las palabras.

In [ ]:
warnings.filterwarnings('ignore')


ldamodel = LdaModel(corpus=corpus, num_topics=10, id2word=diccionario, iterations=500)
pprint(ldamodel.print_topics())

In [ ]:
ldamodel[corpus[4]]

In [ ]:
ldamodel[corpus[0]]

In [ ]:
gensim.matutils.corpus2dense(ldamodel[corpus], ldamodel.num_topics).T

### Visualización de los temas  
Podemos visualizarlo gráficamente la distribución de los documentos del Corpus por temas con la librería `pyLDAvis`

In [ ]:
vis_data = gensimvis.prepare(ldamodel, corpus, diccionario)
pyLDAvis.display(vis_data)

Podemos ver que la separación de temas no es muy buena porque hay algunas palabras muy frecuentes que aparecen en todos los temas. Podemos filtrar estas palabras antes de realizar el LDA del Corpus mediante el método `filter_extremes` de la clase `Dictionary`:

In [ ]:
diccionario.filter_extremes(no_above=0.7) #filtramos las palabras que aparecen en más del 70% de los documentos

In [ ]:
len(diccionario.token2id)

Volvemos a calcular la matriz LDA del Corpus y la representamos gráficamente para ver si es más expresiva

In [ ]:
# Crea corpus (BoW)
textos_trigramas = map(make_trigrams, build_texts(lee_data_file))
corpus = [diccionario.doc2bow(text) for text in textos_trigramas]



In [ ]:
print(corpus[0])

In [ ]:
len(corpus[0])

In [ ]:
# Aplica el modelo LDA
ldamodel = LdaModel(corpus=corpus, num_topics=4, id2word=diccionario, iterations=500)

# Representa gráficamente
vis_data = gensimvis.prepare(ldamodel, corpus, diccionario)
pyLDAvis.display(vis_data)

## Selección del número de temas
Para seleccionar el número óptimo de temas, debemos hacer un barrido y seleccionar el modelo con mayor valor de coherencia (Topic coherence).  
Lo podemos automatizar en una función (nota: *tarda bastante en ejecutarse*).

In [ ]:
def evaluate_graph(dictionary, corpus, texts, limit, start=1, step=1):
    """
    Function to display num_topics - LDA graph using c_v coherence
    
    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : topic limit
    start: min number of topics
    step: step between topics number swept
    
    Returns:
    -------
    lm_list : List of LDA topic models
    c_v : Coherence values corresponding to the LDA model with respective number of topics
    """
    c_v = []
    lm_list = []
    n_topics = list(range(start, limit, step))
    for num_topics in n_topics:
        lm = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        lm_list.append(lm)
        cm = CoherenceModel(model=lm, texts=texts, dictionary=dictionary, coherence='c_v')
        c_v.append(cm.get_coherence())
    
    return lm_list, c_v, n_topics

In [ ]:
textos_trigramas = list(map(make_trigrams, build_texts(lee_data_file)))

lmlist, c_v, n = evaluate_graph(dictionary=diccionario, corpus=corpus, texts=textos_trigramas, limit=20, step=2)

In [ ]:
c_v

### Determinar el tema dominante en cada documento
Una aplicación práctica del topic modeling es determinar de qué tema trata un documento.  
Para hacer esto, se busca el número de tema que tiene una mayor contribución en el documento.  
La función `format_topics_sentences()` genera esta información en forma de tabla.  

In [ ]:
def format_topics_sentences(ldamodel, corpus):
    # inicializa salida
    sent_topics = []

    # obtiene main topic de cada documento
    for row in ldamodel[corpus]:
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        (topic_num, prop_topic)=row[0]
        wp = ldamodel.show_topic(topic_num)
        topic_keywords = ", ".join([word for word, prop in wp])
        sent_topics.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]))
    sent_topics_df = pd.DataFrame(sent_topics)
    sent_topics_df.reset_index(inplace=True)
    sent_topics_df.columns = ['No_doc','Tema_dominante', 'Contribucion_per', 'Palabras_clave']
    sent_topics_df['Tema_dominante'] = sent_topics_df['Tema_dominante'].astype('int')
    return(pd.DataFrame(sent_topics_df))

In [ ]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamodel, corpus=corpus)

df_topic_sents_keywords.head(10)

In [ ]:
ldamodel[corpus[0]]

In [ ]:
ldamodel[corpus[37]]

### Determinar el documento más representativo de cada tema
Agrupando por temas, podemos seleccionar el de mayor porcentaje como más representativo.

In [ ]:
# Agrupamos documentos por tema
sent_topics_sorted = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Tema_dominante')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorted = pd.concat([sent_topics_sorted, 
                                             grp.sort_values(['Contribucion_per'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorted.reset_index(drop=True, inplace=True)

# cambiamos nombre de columna
sent_topics_sorted.columns = df_topic_sents_keywords.columns

# Mostramos
sent_topics_sorted

### Distribución de temas entre documentos
Por último, podemos analizar el volumen y la distribución de temas entre los documentos del tema.  

In [ ]:
# núm de documentos por cada tema
topic_counts = df_topic_sents_keywords['Tema_dominante'].value_counts()

# porcentaje de documentos por cada tema
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# palabras clave de cada tema
topic_num_keywords = sent_topics_sorted[['Tema_dominante', 'Palabras_clave']]

# Concatenamos por columna
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# cambiamos nombre de columna
df_dominant_topics.columns = ['Tema_dominante', 'Palabras_clave', 'Num_Documentos', 'Perc_Documentos']

# Show
df_dominant_topics

### Estimación de temáticas para un documento nuevo
Procesamos el nuevo documento por todo el flujo

In [ ]:
noticia = "A bushfire broke out early this morning in the rural community of \
    Myall Creek, New South Wales, prompting local authorities to issue evacuation \
    warnings. The fire, fueled by dry conditions and strong winds, has scorched \
    approximately 50 hectares of farmland. Emergency services are on site, battling \
    the blaze with both ground crews and aerial firefighting units"

noticia_bow = diccionario.doc2bow(make_trigrams(lemmatize_doc(noticia)))
noticia_topics = ldamodel[noticia_bow]

In [ ]:
noticia_topics

In [ ]:
sorted(noticia_topics, key=lambda x: (x[1]), reverse=True)